In [1]:
# Standard Python modules
import os, sys
import yaml
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib import rcParams
from matplotlib.colors import ListedColormap
import matplotlib.ticker as mticker
# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean
# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes

# Extra 
from scipy.ndimage import gaussian_filter    # smoothing contour lines
from scipy import stats

import metpy.calc as mpcalc
from metpy.units import units
from IPython.display import Image, display

# import personal modules

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap
from timeseries import persistence, select_months, create_list_all_dates
from teleconnections import build_teleconnection_df, mjo_index
from statistical_tests import build_zscore_df, ttest_1samp_new

In [2]:
# Set up paths

path_to_data = '/home/nash/DATA/data/'                            # project data -- read only
path_to_out  = '/home/nash/DATA/repositories/AR_types/out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '/home/nash/DATA/repositories/AR_types/figs/'      # figures

# USE pandas.options TO DISPLAY FLOATS TO TWO DECIMAL PLACES
pd.options.display.float_format = "{:,.2f}".format

In [3]:
ceofid = 'IVT'
ssn ='djfmam'
neofs = 2 # choose number of eofs
nk = 3 # choose number of clusters

# import configuration file for plot dictionary
yaml_doc = '../data/plt_config.yml'
config = yaml.load(open(yaml_doc), Loader=yaml.SafeLoader)
ar_dict = config[ssn]

In [4]:
def ar_daily_df(ssn, nk):
    out_path = path_to_out + ceofid + '/' + ssn + '/' + 'neof' + str(neofs) + '/k' + str(nk) + '/'
    filepath = out_path + 'AR-types_ALLDAYS.csv'
    df = pd.read_csv(filepath)

    # set up datetime index
    df = df.rename(columns={'Unnamed: 0': 'date'})
    df = df.set_index(pd.to_datetime(df.date))
    
    ## Break up columns into different AR Types
    keys = []
    for k in range(nk):
        keys.append("AR_CAT{:1d}".format(k+1,))

    values = np.zeros((len(df.index)))
    dicts = dict(zip(keys, values))

    df_cat = pd.DataFrame(dicts, index=df.index)

    for k in range(nk):
        idx = (df['AR_CAT'] == k+1)
        col = "AR_CAT{:1d}".format(k+1,)
        df_cat.loc[idx, col] = 1
        
    # get total of all AR types
    df_cat['AR_ALL'] = df_cat['AR_CAT1'] + df_cat['AR_CAT2'] + df_cat['AR_CAT3']
    df_cat['AR_CAT'] = df['AR_CAT']
    
    return df_cat

In [5]:
# tele index
start_date = '1979-12-01'
end_date = '2019-05-31'

tele = build_teleconnection_df('daily', 'COND', start_date, end_date)
mjo = mjo_index('daily', ar_dict['start_date'], ar_dict['end_date'])
tele['MJO'] = mjo['COND']
df_index_djfmam = select_months(tele, 12, 5)

# AR df
df_djfmam = ar_daily_df(ssn='djfmam', nk=3)

# combine ar df with tele df
print(len(df_djfmam), len(df_index_djfmam))
# join indices with AR count
new_djfmam = df_djfmam.join(df_index_djfmam)
new_djfmam

7290 7290


,AR_CAT1,AR_CAT2,AR_CAT3,AR_ALL,AR_CAT,AO,PDO,ENSO,SH,MJO
date,,,,,,,,,,
1979-12-01 09:00:00,0.00,1.00,0.00,1.00,2,1,0,0,0,1
1979-12-02 09:00:00,1.00,0.00,0.00,1.00,1,1,0,0,0,1
1979-12-03 09:00:00,0.00,0.00,0.00,0.00,0,1,0,0,0,1
1979-12-04 09:00:00,0.00,0.00,0.00,0.00,0,1,0,0,0,1
1979-12-05 09:00:00,0.00,0.00,0.00,0.00,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2019-05-27 09:00:00,0.00,0.00,0.00,0.00,0,0,0,1,1,1
2019-05-28 09:00:00,0.00,0.00,0.00,0.00,0,-1,0,1,1,1
2019-05-29 09:00:00,1.00,0.00,0.00,1.00,1,-1,0,1,1,1


### Run zscore difference in proportion tests

In [6]:
zscore_djfmam, pval_djfmam = build_zscore_df(new_djfmam)
zscore_djfmam

AR Type,AR Type 1,AR Type 2,AR Type 3,All AR days
teleconnection,,,,
AO+ vs. AO-,2.45*,5.24**,-0.77,4.52**
AO+ vs. Neutral,3.18**,3.28**,-2.91**,1.99*
AO- vs. Neutral,0.69,-2.02*,-2.1*,-2.54*
El Nino vs. La Nina,2.74**,3.97**,0.96,5.07**
El Nino vs. Neutral,0.93,0.07,-0.78,0.03
La Nina vs. Neutral,-2.29*,-4.46**,-1.89,-5.89**
MJO vs Neutral,-1.36,1.08,-0.75,-0.79
MJO vs No MJO,1.35,-1.54,-0.7,-0.71
No MJO vs Neutral,-2.19*,1.98*,-0.26,-0.3


In [7]:
pval_djfmam

AR Type,AR Type 1,AR Type 2,AR Type 3,All AR days
teleconnection,,,,
AO+ vs. AO-,0.01,0.00,0.44,0.00
AO+ vs. Neutral,0.00,0.00,0.00,0.05
AO- vs. Neutral,0.49,0.04,0.04,0.01
El Nino vs. La Nina,0.01,0.00,0.34,0.00
El Nino vs. Neutral,0.35,0.95,0.43,0.97
La Nina vs. Neutral,0.02,0.00,0.06,0.00
MJO vs Neutral,0.17,0.28,0.45,0.43
MJO vs No MJO,0.18,0.12,0.48,0.48
No MJO vs Neutral,0.03,0.05,0.79,0.77


In [8]:
tmp = zscore_djfmam
print(tmp.to_latex(index=True))  

\begin{tabular}{lllll}
\toprule
AR Type & AR Type 1 & AR Type 2 & AR Type 3 & All AR days \\
teleconnection      &           &           &           &             \\
\midrule
AO+ vs. AO-         &     2.45* &    5.24** &     -0.77 &      4.52** \\
AO+ vs. Neutral     &    3.18** &    3.28** &   -2.91** &       1.99* \\
AO- vs. Neutral     &      0.69 &    -2.02* &     -2.1* &      -2.54* \\
El Nino vs. La Nina &    2.74** &    3.97** &      0.96 &      5.07** \\
El Nino vs. Neutral &      0.93 &      0.07 &     -0.78 &        0.03 \\
La Nina vs. Neutral &    -2.29* &   -4.46** &     -1.89 &     -5.89** \\
MJO vs Neutral      &     -1.36 &      1.08 &     -0.75 &       -0.79 \\
MJO vs No MJO       &      1.35 &     -1.54 &      -0.7 &       -0.71 \\
No MJO vs Neutral   &    -2.19* &     1.98* &     -0.26 &        -0.3 \\
PDO+ vs. Neutral    &   -2.82** &      -1.0 &    3.44** &        0.28 \\
PDO+ vs. PDO-       &     -2.1* &     2.03* &    4.37** &       3.5** \\
PDO- vs. Neutral    & 

In [ ]:
# create df showing number of AR days and years (by type, tele, and cond)

### Bar Plot Fractions

AR frequency is calculated as the percent AR days out of all days in a given AO, ENSO, SH, or PDO phase during December–March

e.g. (number of AR days/number of days in El Nino phase) vs. (number of AR days/# of days in Neutral)

In [ ]:
cols = ['AR_CAT1', 'AR_CAT2', 'AR_CAT3', 'AR_ALL']
djfmam = new_djfmam.drop(columns=cols)
results_test = []
for i, df in enumerate([djfmam]):
    test_df = df
    print(len(test_df))
    # count the frequency of each AR type based on phase conditions
    test = test_df.groupby(['AR_CAT']).agg({'AO': 'value_counts',
                                        'PDO': 'value_counts',
                                        'ENSO': 'value_counts',
                                        'SH': 'value_counts'})
    print(test)
    test3 = test_df.agg({'AO': 'value_counts',
                                        'PDO': 'value_counts',
                                        'ENSO': 'value_counts',
                                        'SH': 'value_counts'})
    test3 = pd.concat([test3]*4, keys=[0, 1, 2, 3])
    print(test3)
    # get the frequency value
    test2 = test.div(test3, axis=0)
    print(test2)

    # put into a new dataframe and reset the index
    data=pd.DataFrame(test2*100)
    data = data.reset_index()

    # melt the dataframe to make it tidy
    results = pd.melt(data, id_vars=['AR_CAT', 'level_1'], value_vars=['AO', 'PDO', 'ENSO', 'SH'],
                      var_name='teleconnection', value_name='frequency')

    # drop rows where level_0 = 0
    # or where there is no AR
    results_test.append(results[results.AR_CAT != 0])

In [ ]:
# TO DO: add pvalues to x-axis labels
fname = path_to_figs + 'zscore_barplot'
fmt = 'png'
fig = plt.figure()
fig.set_size_inches((8.5,2.0))
fig.dpi = 300
nrows = 1
ncols = 3
nplots=nrows*ncols

artype = [1, 2, 3]
typ_label = ['AR Type 1', 'AR Type 2', 'AR Type 3']
flatui = ['c', 'grey', 'red', ]
row_lbl1 = ['DJFMAM (%)', '', '']

sns.set_style("whitegrid")

#### DJFMAM Plots
for i, (p, ar) in enumerate(zip(np.arange(0, 3), artype)):
    sns.set_style("whitegrid")
    data = results_test[0]
    ax = plt.subplot(nrows, ncols, p+1)
    df = data[data.AR_CAT == ar]
    g = sns.barplot(x=df['teleconnection'], y=df['frequency'], hue=df["level_1"],
                data=df, palette=flatui, dodge=True, ci=None)
#     for p in g.patches[:4]:
#         g.annotate("$p$=" + format(p.get_height(), '.1f'), 
#                    (p.get_x() + p.get_width() / 2., p.get_height()), 
#                    ha = 'center', va = 'center',
#                    size=6,
#                    xytext = (0, 9), 
#                    textcoords = 'offset points')
    
    g.legend_.remove()
    sns.set_style("ticks")
    ax.set_ylabel(row_lbl1[i], fontsize=12)
    ax.set_xlabel('')
    plt.ylim(0, 15)
    ax.set_title(typ_label[i], loc='center')

# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., labels=['Negative', 'Neutral', 'Positive'], title='Phase')
    

plt.tight_layout()
# Save the figure
fig.savefig('%s.%s' % (fname, fmt), bbox_inches='tight', dpi=fig.dpi)
fig.clf()

plotFile = fname + '.png'
print(plotFile)
display(Image(plotFile))